In [172]:
# import os
# os.listdir('../input')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, re


In [149]:
path='../input/'
# a = pd.read_csv(f'{path}air_reserve.csv',)
# a = pd.read_csv(f'{path}air_store_info.csv',)
# a = pd.read_csv(f'{path}air_visit_data.csv',)
di = pd.read_csv(f'{path}date_info.csv',)
# a = pd.read_csv(f'{path}hpg_reserve.csv',)
# a = pd.read_csv(f'{path}hpg_store_info.csv',)
# a = pd.read_csv(f'{path}sample_submission.csv',)
# a = pd.read_csv(f'{path}store_id_relation.csv',)


In [92]:
s = pd.read_csv(f'{path}sample_submission.csv',)

s['store_id'] = s['id'].str[:20]; s['visit_date'] = s['id'].str[21:]

s.drop(['visitors'], axis=1, inplace=True)

s['visit_date'] = pd.to_datetime(s['visit_date'])
# 念のためカラム情報の確認
s.info();s.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32019 entries, 0 to 32018
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          32019 non-null  object        
 1   store_id    32019 non-null  object        
 2   visit_date  32019 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 750.6+ KB


,id,store_id,visit_date
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27


In [93]:
s.sort_values('visit_date',ascending=False)
# 予測対象日は2017-04-23-2017-05-31

,id,store_id,visit_date
32018,air_fff68b929994bfbd_2017-05-31,air_fff68b929994bfbd,2017-05-31
5342,air_2fc478dc9f0a6b31_2017-05-31,air_2fc478dc9f0a6b31,2017-05-31
22463,air_b23d0f519291247d_2017-05-31,air_b23d0f519291247d,2017-05-31
9554,air_52e2a1fd42bc917a_2017-05-31,air_52e2a1fd42bc917a,2017-05-31
14195,air_73f316e6a18d8aa9_2017-05-31,air_73f316e6a18d8aa9,2017-05-31
...,...,...,...
24063,air_bf7591560077332d_2017-04-23,air_bf7591560077332d,2017-04-23
7995,air_457efe8c3a30ea17_2017-04-23,air_457efe8c3a30ea17,2017-04-23
24024,air_bf617aa68d5f1cfa_2017-04-23,air_bf617aa68d5f1cfa,2017-04-23
8034,air_464a62de0d57be1e_2017-04-23,air_464a62de0d57be1e,2017-04-23


In [94]:
s['visit_date'] = pd.to_datetime(s.visit_date)
s['dow'] = s['visit_date'].dt.dayofweek
s


,id,store_id,visit_date,dow
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,6
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,1
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,2
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,3
...,...,...,...,...
32014,air_fff68b929994bfbd_2017-05-27,air_fff68b929994bfbd,2017-05-27,5
32015,air_fff68b929994bfbd_2017-05-28,air_fff68b929994bfbd,2017-05-28,6
32016,air_fff68b929994bfbd_2017-05-29,air_fff68b929994bfbd,2017-05-29,0
32017,air_fff68b929994bfbd_2017-05-30,air_fff68b929994bfbd,2017-05-30,1


In [166]:
v = pd.read_csv(f'{path}air_visit_data.csv',)

In [96]:
print(v.shape)
v.head()

(252108, 3)


,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [97]:
# v.sort_values('visit_date',ascending=False)
# 範囲は2016-01-01-2017-04-22
# v.reserve_visitors = v.reserve_visitors.astype('float64')
v['visit_date'] = pd.to_datetime(v.visit_date)
v.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   air_store_id  252108 non-null  object        
 1   visit_date    252108 non-null  datetime64[ns]
 2   visitors      252108 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 5.8+ MB


In [98]:
va = v[v['air_store_id']=='air_00a91d42b08b08d9']
va.describe()


,visitors
count,232.000000
mean,26.081897
std,12.435364
min,1.000000
25%,18.000000
50%,26.000000
75%,34.000000
max,99.000000


In [99]:
va.visit_date.describe()

<ipython-input-99-2939c70a48d6>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  va.visit_date.describe()


count                     232
unique                    232
top       2016-08-08 00:00:00
freq                        1
first     2016-07-01 00:00:00
last      2017-04-22 00:00:00
Name: visit_date, dtype: object

In [100]:
v['dow'] = v['visit_date'].dt.dayofweek

In [101]:
display('VVVVVVVVVVVV',v)
display('SSSSSSSSSSSS',s)

'VVVVVVVVVVVV'

,air_store_id,visit_date,visitors,dow
0,air_ba937bf13d40fb24,2016-01-13,25,2
1,air_ba937bf13d40fb24,2016-01-14,32,3
2,air_ba937bf13d40fb24,2016-01-15,29,4
3,air_ba937bf13d40fb24,2016-01-16,22,5
4,air_ba937bf13d40fb24,2016-01-18,6,0
...,...,...,...,...
252103,air_24e8414b9b07decb,2017-04-18,6,1
252104,air_24e8414b9b07decb,2017-04-19,6,2
252105,air_24e8414b9b07decb,2017-04-20,7,3
252106,air_24e8414b9b07decb,2017-04-21,8,4


'SSSSSSSSSSSS'

,id,store_id,visit_date,dow
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,6
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,1
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,2
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,3
...,...,...,...,...
32014,air_fff68b929994bfbd_2017-05-27,air_fff68b929994bfbd,2017-05-27,5
32015,air_fff68b929994bfbd_2017-05-28,air_fff68b929994bfbd,2017-05-28,6
32016,air_fff68b929994bfbd_2017-05-29,air_fff68b929994bfbd,2017-05-29,0
32017,air_fff68b929994bfbd_2017-05-30,air_fff68b929994bfbd,2017-05-30,1


In [102]:
aggregation = {'visitors' : 'median'}
vagg = v.groupby(['air_store_id','dow']).agg(aggregation).reset_index()
vagg

,air_store_id,dow,visitors
0,air_00a91d42b08b08d9,0,19.0
1,air_00a91d42b08b08d9,1,24.5
2,air_00a91d42b08b08d9,2,28.0
3,air_00a91d42b08b08d9,3,30.0
4,air_00a91d42b08b08d9,4,35.5
...,...,...,...
5736,air_fff68b929994bfbd,2,4.0
5737,air_fff68b929994bfbd,3,4.0
5738,air_fff68b929994bfbd,4,5.0
5739,air_fff68b929994bfbd,5,7.0


In [103]:
merged = pd.merge(s, vagg, how='left', left_on=['store_id', 'dow'], right_on=['air_store_id', 'dow'])
 
# idとvisitorsだけをsubmitへ格納
submit = merged[['id', 'visitors']]
 
# submitのヘッダー情報
missing_values_table(submit)

Your selected dataframe has 2 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
visitors,341,1.1


In [104]:
submit.fillna(0, inplace=True)
# 念のため確認
missing_values_table(submit)

Your selected dataframe has 2 columns.
There are 0 columns that have missing values.


C:\Users\hayatotominaga\anaconda3\envs\rake4\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,Missing Values,% of Total Values


In [105]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns


重み付き平均（加重平均）を算出  
「曜日」と「祝日フラグ」  
前処理  
- 週末の場合（土曜日or日曜日）は、祝日であってもフラグを0とする処理を行いましょう。
- 日付を基に「重み」を作成
    - 日付が古いものには少ない重みを、予測する日付に近い（新しい日付）には多い重み
- air_visit_data(Airレジの各レストランの実客数データ）に上記で算出したWeight（重み）を加えて、visitors（実客数）をnp.log1pを使って対数にして、「air_store_id」「day_of_week」「holiday_flg」（各レストランID、曜日、祝日フラグ）でグルーピングをして 重み付き平均を算出します。

In [157]:
# axis{0 or ‘index’, 1 or ‘columns’}, default 0
# 0 or ‘index’: apply function to each column.
weekend_hdays = di.apply((lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
weekend_hdays
# # date_infoの該当の箇所のフラグを1から0へ更新をする
di.loc[weekend_hdays, 'holiday_flg'] = 0
 
 

In [163]:
di['weight'] = (di.index + 1) / len(di) 

,calendar_date,day_of_week,holiday_flg,weight
512,2017-05-27,Saturday,0,0.992263
513,2017-05-28,Sunday,0,0.994197
514,2017-05-29,Monday,0,0.996132
515,2017-05-30,Tuesday,0,0.998066
516,2017-05-31,Wednesday,0,1.000000


In [169]:
v

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
...,...,...,...
252103,air_24e8414b9b07decb,2017-04-18,6
252104,air_24e8414b9b07decb,2017-04-19,6
252105,air_24e8414b9b07decb,2017-04-20,7
252106,air_24e8414b9b07decb,2017-04-21,8


In [174]:
vd = v.merge(di, left_on='visit_date', right_on='calendar_date', how='left')
vd.drop('calendar_date', axis=1, inplace=True)
 
# vdの実客数にnp.log1pの対数関数を使って処理
vd['visitors'] = vd.visitors.map(np.log1p)
 
# vdの確認
vd.head(10)
 

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,weight
0,air_ba937bf13d40fb24,2016-01-13,3.258097,Wednesday,0,0.025145
1,air_ba937bf13d40fb24,2016-01-14,3.496508,Thursday,0,0.027079
2,air_ba937bf13d40fb24,2016-01-15,3.401197,Friday,0,0.029014
3,air_ba937bf13d40fb24,2016-01-16,3.135494,Saturday,0,0.030948
4,air_ba937bf13d40fb24,2016-01-18,1.945910,Monday,0,0.034816
5,air_ba937bf13d40fb24,2016-01-19,2.302585,Tuesday,0,0.036750
6,air_ba937bf13d40fb24,2016-01-20,3.465736,Wednesday,0,0.038685
7,air_ba937bf13d40fb24,2016-01-21,3.091042,Thursday,0,0.040619
8,air_ba937bf13d40fb24,2016-01-22,2.944439,Friday,0,0.042553
9,air_ba937bf13d40fb24,2016-01-23,3.295837,Saturday,0,0.044487


In [175]:
wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
 
 
# グルーピングして重み付き平均を算出
visitors = vd.groupby(
    ['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) 
 
# データを確認
visitors.head(10)

,air_store_id,day_of_week,holiday_flg,visitors
0,air_00a91d42b08b08d9,Friday,0,3.590026
1,air_00a91d42b08b08d9,Monday,0,3.091672
2,air_00a91d42b08b08d9,Monday,1,3.091042
3,air_00a91d42b08b08d9,Saturday,0,2.526836
4,air_00a91d42b08b08d9,Sunday,0,1.098612
5,air_00a91d42b08b08d9,Thursday,0,3.414904
6,air_00a91d42b08b08d9,Tuesday,0,3.173611
7,air_00a91d42b08b08d9,Wednesday,0,3.330028
8,air_0164b9927d20bcc3,Friday,0,2.327400
9,air_0164b9927d20bcc3,Friday,1,2.747824


In [177]:
s

,id,store_id,visit_date,dow,air_store_id,calendar_date
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,6,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,1,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,2,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,3,air_00a91d42b08b08d9,2017-04-27
...,...,...,...,...,...,...
32014,air_fff68b929994bfbd_2017-05-27,air_fff68b929994bfbd,2017-05-27,5,air_fff68b929994bfbd,2017-05-27
32015,air_fff68b929994bfbd_2017-05-28,air_fff68b929994bfbd,2017-05-28,6,air_fff68b929994bfbd,2017-05-28
32016,air_fff68b929994bfbd_2017-05-29,air_fff68b929994bfbd,2017-05-29,0,air_fff68b929994bfbd,2017-05-29
32017,air_fff68b929994bfbd_2017-05-30,air_fff68b929994bfbd,2017-05-30,1,air_fff68b929994bfbd,2017-05-30


In [183]:
ss = pd.read_csv(f'{path}sample_submission.csv',)

ss['store_id'] = ss['id'].str[:20]; ss['visit_date'] = ss['id'].str[21:]
# ss.drop(['visitors'], axis=1, inplace=True)

ss['visit_date'] = pd.to_datetime(ss['visit_date'])


In [184]:
ss

,id,visitors,store_id,visit_date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27
...,...,...,...,...
32014,air_fff68b929994bfbd_2017-05-27,0,air_fff68b929994bfbd,2017-05-27
32015,air_fff68b929994bfbd_2017-05-28,0,air_fff68b929994bfbd,2017-05-28
32016,air_fff68b929994bfbd_2017-05-29,0,air_fff68b929994bfbd,2017-05-29
32017,air_fff68b929994bfbd_2017-05-30,0,air_fff68b929994bfbd,2017-05-30


In [214]:

ss['air_store_id'] = ss.id.map(
    lambda x: '_'.join(x.split('_')[:-1]))
ss

,id,visitors,store_id,visit_date,air_store_id
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,air_00a91d42b08b08d9
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,air_00a91d42b08b08d9
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,air_00a91d42b08b08d9
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,air_00a91d42b08b08d9
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,air_00a91d42b08b08d9
...,...,...,...,...,...
32014,air_fff68b929994bfbd_2017-05-27,0,air_fff68b929994bfbd,2017-05-27,air_fff68b929994bfbd
32015,air_fff68b929994bfbd_2017-05-28,0,air_fff68b929994bfbd,2017-05-28,air_fff68b929994bfbd
32016,air_fff68b929994bfbd_2017-05-29,0,air_fff68b929994bfbd,2017-05-29,air_fff68b929994bfbd
32017,air_fff68b929994bfbd_2017-05-30,0,air_fff68b929994bfbd,2017-05-30,air_fff68b929994bfbd


In [215]:

ss['calendar_date'] = ss.id.map(lambda x: x.split('_')[2])
 
# 重み付き平均で予測したvisitorsとsをマージする
ss.drop('visitors', axis=1, inplace=True)
ss = ss.merge(di, on='calendar_date', how='left')
ss = ss.merge(
    visitors, on=['air_store_id', 'day_of_week', 'holiday_flg'], how='left')
 
# データセットを確認してみよう
ss.head()

,id,store_id,visit_date,air_store_id,calendar_date,day_of_week,holiday_flg,weight,visitors
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,air_00a91d42b08b08d9,2017-04-23,Sunday,0,0.926499,1.098612
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,air_00a91d42b08b08d9,2017-04-24,Monday,0,0.928433,3.091672
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,0.930368,3.173611
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,0.932302,3.330028
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,air_00a91d42b08b08d9,2017-04-27,Thursday,0,0.934236,3.414904


In [216]:
missing_values_table(ss)

Your selected dataframe has 9 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
visitors,668,2.1


In [217]:
missings = ss.visitors.isnull()
ss.loc[missings, 'visitors'] = ss[missings].merge(
    visitors[visitors.holiday_flg==0], on=(
        'air_store_id', 'day_of_week'), how='left')['visitors_y'].values
 
# 改めて欠損データの確認
missing_values_table(ss)

Your selected dataframe has 9 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
visitors,448,1.4


In [218]:
missings = ss.visitors.isnull()
ss.loc[missings, 'visitors'] = ss[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values
 
# 欠損データを確認
missing_values_table(ss)

Your selected dataframe has 9 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [222]:
ss['visitors'] = ss.visitors.map(np.expm1)
 
# 提出フォーマットの規定に合うように処理してsub_fileへ格納
ss = ss[['id', 'visitors']]
ss['visitors'][ss['visitors'] ==0] = ss['visitors'][ss['visitors'] ==0]
sub_file = ss.copy()
 
# データの確認
sub_file.head()
 

C:\Users\hayatotominaga\anaconda3\envs\rake4\lib\site-packages\pandas\core\base.py:937: RuntimeWarning: overflow encountered in expm1
  new_values = map_f(values, mapper)
<ipython-input-222-9a15c4f3069f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss['visitors'][ss['visitors'] ==0] = ss['visitors'][ss['visitors'] ==0]


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,594.294415
1,air_00a91d42b08b08d9_2017-04-24,inf
2,air_00a91d42b08b08d9_2017-04-25,inf
3,air_00a91d42b08b08d9_2017-04-26,inf
4,air_00a91d42b08b08d9_2017-04-27,inf


In [66]:
a = pd.read_csv(f'{path}hpg_reserve.csv',)


In [67]:
a.dtypes

hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors     int64
dtype: object

In [68]:
a['date'] = pd.to_datetime(a.visit_datetime); a.set_index("date",inplace=True)

In [70]:
a = a.sort_index();a
a

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
date,,,,
2016-01-01 11:00:00,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
2016-01-01 13:00:00,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2016-01-01 16:00:00,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
2016-01-01 17:00:00,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
2016-01-01 17:00:00,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13
...,...,...,...,...
2017-05-31 20:00:00,hpg_2b293477ea4642fb,2017-05-31 20:00:00,2017-04-19 07:00:00,2
2017-05-31 20:00:00,hpg_f922aeca48ca5a29,2017-05-31 20:00:00,2017-03-19 00:00:00,3
2017-05-31 21:00:00,hpg_e9151de687b93aa3,2017-05-31 21:00:00,2017-04-14 20:00:00,49


In [84]:
a.groupby(['hpg_store_id','visit_datetime']).agg(sum)

reserve_visitors
hpg_store_id         visit_datetime                       
hpg_001112ef76b9802c 2016-02-26 19:00:00                 9
                     2016-03-17 19:00:00                 3
                     2016-03-31 18:00:00                 5
                     2016-04-05 18:00:00                13
                     2016-04-18 17:00:00                 9
...                                                    ...
hpg_fffc097dce87af3e 2017-03-10 19:00:00                 2
                     2017-03-17 19:00:00                 2
                     2017-03-27 19:00:00                 5
                     2017-04-21 18:00:00                 3
                     2017-04-23 18:00:00                 8

[1711380 rows x 1 columns]

In [81]:
a.reset_index(inplace=True)
a.sort_values('hpg_store_id')

,date,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
731442,2016-09-07 18:00:00,hpg_001112ef76b9802c,2016-09-07 18:00:00,2016-08-22 10:00:00,5
1976775,2017-04-27 18:00:00,hpg_001112ef76b9802c,2017-04-27 18:00:00,2017-04-22 22:00:00,15
1969937,2017-04-24 19:00:00,hpg_001112ef76b9802c,2017-04-24 19:00:00,2017-04-19 04:00:00,1
830059,2016-10-06 19:00:00,hpg_001112ef76b9802c,2016-10-06 19:00:00,2016-10-02 19:00:00,22
1203285,2016-12-20 19:00:00,hpg_001112ef76b9802c,2016-12-20 19:00:00,2016-12-12 18:00:00,4
...,...,...,...,...,...
1810370,2017-03-27 19:00:00,hpg_fffc097dce87af3e,2017-03-27 19:00:00,2017-03-22 14:00:00,5
1946673,2017-04-21 18:00:00,hpg_fffc097dce87af3e,2017-04-21 18:00:00,2017-04-18 21:00:00,3
924278,2016-10-30 19:00:00,hpg_fffc097dce87af3e,2016-10-30 19:00:00,2016-10-20 20:00:00,13
698030,2016-08-26 19:00:00,hpg_fffc097dce87af3e,2016-08-26 19:00:00,2016-08-23 18:00:00,3


In [80]:
a.pivot(index='visit_datetime', columns='hpg_store_id', values='reserve_visitors')

ValueError: Index contains duplicate entries, cannot reshape